# Comparing Sampling Generation vs. Regular Generation in LLMs

This notebook explores the benefits and differences between sampling generation and regular generation using Large Language Models (LLMs). We will demonstrate how generating multiple candidates with a cheaper model and refining the final result can lead to improved quality compared to generating results directly using a chain of thought.

## Table of Contents

1. [Introduction](#introduction)
2. [Setup](#setup)
    - [Import Libraries](#import-libraries)
    - [Load Environment Variables](#load-environment-variables)
    - [Define Constants](#define-constants)
3. [Data Loading and Preparation](#data-loading-and-preparation)
    - [Load and Split Documents](#load-and-split-documents)
4. [Initialize Models and Embeddings](#initialize-models-and-embeddings)
5. [Sampling Generation Approach](#sampling-generation-approach)
    - [Generate Questions Using Sampling](#generate-questions-using-sampling)
6. [Regular Generation Approach](#regular-generation-approach)
    - [Generate Questions Using Regular Method](#generate-questions-using-regular-method)
7. [Comparative Analysis](#comparative-analysis)
    - [Quality of Generated Questions](#quality-of-generated-questions)
    - [Cost Efficiency](#cost-efficiency)
8. [Conclusion](#conclusion)

## Introduction

In the context of question generation using LLMs, two primary methods can be employed:

1. **Sampling Generation**: Generate a large number of candidate questions using a cheaper model and then refine these to produce the final set.
2. **Regular Generation**: Directly generate questions using a more advanced model with chain-of-thought reasoning.

This notebook compares these two methods in terms of quality and efficiency.

## Setup

### Import Libraries

In [15]:
import random
import copy
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from synthlume.pipeline.step import (
    GenerateQuestionThinkingProcess,
    GenerateQuestionWithEnhancedContextStep,
    GenerateQuestionFromSamples,
)
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import PyPDFLoader

### Load Environment Variables

In [16]:
# Load Azure OpenAI credentials from environment variables
AZURE_OPENAI_KEY = os.getenv("AZURE_OPENAI_KEY")
AZURE_DEPLOYMENT_NAME = os.getenv("AZURE_DEPLOYMENT_NAME")
AZURE_ENDPOINT = os.getenv("AZURE_ENDPOINT")

### Define Constants

In [17]:
# Define paths and constants
BASE_PATH = "../data/moodys_data"
CHUNK_SIZE = 2048
CHUNK_OVERLAP = 256
SEED = 27
MAX_STEPS = 5

## Data Loading and Preparation

### Load and Split Documents

In [19]:
def load_and_split(path, splitter):
    loader = PyPDFLoader(path)
    documents = loader.load()
    return splitter.split_documents(documents)

# Get list of PDF files
pdfs = [
    os.path.join(BASE_PATH, filename)
    for filename in os.listdir(BASE_PATH)
    if filename.endswith(".pdf")
]

# Initialize text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=CHUNK_SIZE,
    chunk_overlap=CHUNK_OVERLAP,
    length_function=len,
    is_separator_regex=False,
)

# Load and split all documents
all_documents = sum([load_and_split(pdf, text_splitter) for pdf in pdfs], [])

## Initialize Models and Embeddings

In [20]:
# Initialize embeddings
embeddings = AzureOpenAIEmbeddings(
    openai_api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    azure_deployment="text-embedding-ada-002",
    openai_api_version="2024-09-01-preview",
)

# Initialize LLMs
llm_large = AzureChatOpenAI(
    openai_api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    openai_api_version="2024-09-01-preview",
    deployment_name=AZURE_DEPLOYMENT_NAME,
    temperature=0.9,
)

llm_small = AzureChatOpenAI(
    openai_api_key=AZURE_OPENAI_KEY,
    azure_endpoint=AZURE_ENDPOINT,
    openai_api_version="2024-09-01-preview",
    deployment_name="gpt-4o-mini",
    temperature=0.9,
)

## Sampling Generation Approach

In this approach, we use a smaller, cheaper LLM to generate multiple candidate questions and then refine them to produce the final questions.


### Generate Questions Using Sampling

In [21]:
# Initialize sampling generation step
sampling_generation = GenerateQuestionThinkingProcess(
    llm=llm_small,
    language="en",
    documents=all_documents,
    embeddings=embeddings,
    n_samples=15,
    n_documents=10,
    min_distance=0.85,
    max_distance=0.98,
)

# Initialize question refinement step
generate_from_samples_step = GenerateQuestionFromSamples(llm=llm_large, language="en")

# Prepare results list
sampling_results = []

# Shuffle documents
random.seed(SEED)
random.shuffle(all_documents)

# Iterate over document chunks
for i, chunk in enumerate(all_documents[:MAX_STEPS]):
    print(f"Processing Chunk {i+1}/{MAX_STEPS} using Sampling Generation")
    try:
        inputs = {
            "context": chunk,
            "current_document": chunk.metadata["source"],
        }

        # Generate samples
        samples_response = sampling_generation.generate(**inputs)

        # Refine samples to generate final questions
        generation = generate_from_samples_step.generate(**samples_response)

        # Collect results
        for option in generation["questions"]:
            question_data = {
                "question": option["question"],
                "answer": option["answer"],
                "context": chunk.page_content,
            }
            sampling_results.append(question_data)

            # Print the question
            print(f"\nQuestion: {option['question']}\n")

    except Exception as e:
        print(f"Error processing chunk {i+1}: {e}")

Processing Chunk 1/5 using Sampling Generation


2024-10-16 12:05:35,103 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 0
2024-10-16 12:05:39,183 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 1
2024-10-16 12:05:43,403 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 2
2024-10-16 12:05:48,114 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 3
2024-10-16 12:05:53,714 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 4
2024-10-16 12:05:58,576 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 5
2024-10-16 12:06:03,883 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 6
2024-10-16 12:06:07,159 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 7
2024-10-16 12:06:11,154 - INFO - synthlume.pipeline.step.generate_question_thinking_proc


Question: How do economic conditions and government policies in emerging markets converge to influence Honda's motorcycle and automotive sales, particularly in the aftermath of the pandemic?


Question: In what ways do government policies and economic volatility in emerging markets interact to influence consumer purchasing behavior for Honda's motorcycle and automotive businesses?


Question: How do economic conditions and government policies in emerging markets impact consumer behavior toward motorcycle purchases and Honda's broader automotive strategy?


Question: How do economic recovery factors, including currency fluctuations, tourism dependency, and government policies, influence consumer demand for Honda's motorcycle and automotive products in emerging markets?


Question: What are the implications of government policies and economic recovery on consumer demand for motorcycles in emerging markets, and how does this relate to the overall automotive business strategy of Honda in 

2024-10-16 12:06:58,920 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 0
2024-10-16 12:07:03,893 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 1
2024-10-16 12:07:12,746 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 2
2024-10-16 12:07:22,228 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 3
2024-10-16 12:07:27,955 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 4
2024-10-16 12:07:32,822 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 5
2024-10-16 12:07:39,529 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 6
2024-10-16 12:07:44,859 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 7
2024-10-16 12:07:52,376 - INFO - synthlume.pipeline.step.generate_question_thinking_proc


Question: How do Tesla's production strategies and reliance on specific vehicle models influence its competitive positioning and financial performance in the face of escalating competition from traditional automotive manufacturers transitioning to electric vehicles?


Question: How does Tesla's competitive positioning within the electric vehicle market, coupled with its production capabilities and regulatory pressures, influence its EBITA margin and overall financial outlook in the face of increasing competition from both traditional automakers and new entrants?


Question: How do Tesla's current product dependency and its competitive market positioning forecast its financial stability and growth potential amid increased competition in the battery electric vehicle sector?


Question: How does Tesla's competitive advantage in the electric vehicle market, particularly its operational efficiency and product lineup, mitigate the risks posed by increasing competition from traditional autom

...


Question: How do the proposed Chinese tariffs impact US agricultural sectors, particularly soybeans, compared to the aerospace and chemicals industries, and what are the broader macroeconomic implications of these tariffs on US-China trade relations?


Question: What are the primary sectors of the US economy that are expected to experience significant disruptions due to the proposed Chinese tariffs, and how might their varying credit impacts reflect their relation to US exports and economic stability?


Question: How might the proposed tariffs on US exports to China, particularly in the agriculture and chemicals sectors, influence credit ratings and overall economic confidence within these industries, particularly considering their historical export volumes?


Question: What are the anticipated macroeconomic implications of the proposed Chinese tariffs on US agriculture, specifically soybeans, in conjunction with the ability of US chemicals producers to adapt their trade flows in resp

2024-10-16 12:10:33,547 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 0
2024-10-16 12:10:37,607 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 1
2024-10-16 12:10:42,313 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 2
2024-10-16 12:10:45,385 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 3
2024-10-16 12:10:48,866 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 4
2024-10-16 12:10:53,171 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 5
2024-10-16 12:10:57,317 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 6
2024-10-16 12:11:00,335 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 7
2024-10-16 12:11:04,125 - INFO - synthlume.pipeline.step.generate_question_thinking_proc


Question: How does the growth in demand for electric vehicle batteries influence the operational profitability and risk management strategies of companies within this sector, particularly considering their leverage and credit ratings?


Question: How do the profitability percentages attributed to the battery business influence the strategic decisions of companies facing the growing demand and operational risks in the electric vehicle battery market?


Question: How do the anticipated increases in operating profits within the electric vehicle battery sector correlate with the operational and investment risks identified amidst growing demand?


Question: How do the operational profit contributions of electric vehicle battery businesses relate to the projected investment risks associated with the increasing demand for these batteries?


Question: How do the rising operational profits in the electric vehicle battery sector have implications on the associated risks and investment requireme

2024-10-16 12:11:59,411 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 0
2024-10-16 12:12:06,255 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 1
2024-10-16 12:12:11,107 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 2
2024-10-16 12:12:16,420 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 3
2024-10-16 12:12:21,638 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 4
2024-10-16 12:12:31,575 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 5
2024-10-16 12:12:39,461 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 6
2024-10-16 12:12:44,975 - INFO - synthlume.pipeline.step.generate_question_thinking_process - Generating sample 7
2024-10-16 12:12:52,752 - INFO - synthlume.pipeline.step.generate_question_thinking_proc


Question: How have Tesla's adjustments of debt and EBITDA over time impacted its debt-to-EBITDA ratio, and what does this imply about its financial health compared to industry peers?


Question: How have Tesla's adjusted EBITDA and total debt evolved over the years, and what impact has this had on its profitability margins as reflected in its financial ratios?


Question: How have Tesla's revenue growth and adjusted EBITDA trends influenced its debt levels and credit rating over the years, particularly between 2017 and 2021?


Question: How did the variations in Tesla's adjusted debt and EBITDA from 2016 to 2021 impact its profitability margins and liquidity ratios?


Question: How have the trends in Tesla’s adjusted debt and EBITDA ratios evolved over the years, and what implications does this have for the company’s financial stability and profitability compared to industry peers?



## Regular Generation Approach

In the regular generation method, we use a larger LLM with chain-of-thought reasoning to generate questions directly.

### Generate Questions Using Regular Method

In [22]:
# Initialize regular generation step
multicontext_generation_step = GenerateQuestionWithEnhancedContextStep(
    llm=llm_large,
    language="en",
    documents=all_documents,
    embeddings=embeddings,
    n_documents=10,
    min_distance=0.85,
    max_distance=0.98,
)

# Prepare results list
regular_results = []

# Iterate over the same document chunks
for i, chunk in enumerate(all_documents[:MAX_STEPS]):
    print(f"Processing Chunk {i+1}/{MAX_STEPS} using Regular Generation")
    try:
        inputs = {
            "context": chunk,
            "current_document": chunk.metadata["source"],
        }

        # Generate questions
        for _ in range(3):  # Generate 3 questions per chunk
            generated_question = multicontext_generation_step.generate(**inputs)
            regular_results.append(generated_question)

            # Print the question
            print(f"\nQuestion: {generated_question['question']}\n")

    except Exception as e:
        print(f"Error processing chunk {i+1}: {e}")

Processing Chunk 1/5 using Regular Generation


2024-10-16 12:13:52,522 - INFO - synthlume.pipeline.step.generate_question_with_enhanced_context - Most similar scores: [0.9999971329818891, 0.7950722277164459, 0.7447011172771454, 0.7413454055786133, 0.7405601143836975, 0.7296861410140991, 0.7282407283782959, 0.7228216826915741, 0.7191706299781799, 0.7166908979415894, 0.715585321187973, 0.7137781381607056, 0.7111849784851074, 0.7066434323787689, 0.7050520181655884, 0.7050461173057556, 0.6996228098869324, 0.6987706124782562, 0.6977443695068359, 0.6976523697376251]



Question: How have economic volatility, weakened currencies, and government policies in emerging markets impacted Honda's motorcycle sales and, by extension, its automotive business during the pandemic?



2024-10-16 12:13:59,468 - INFO - synthlume.pipeline.step.generate_question_with_enhanced_context - Most similar scores: [0.9999971329818891, 0.7950722277164459, 0.7447011172771454, 0.7413454055786133, 0.7405601143836975, 0.7296861410140991, 0.7282407283782959, 0.7228216826915741, 0.7191706299781799, 0.7166908979415894, 0.715585321187973, 0.7137781381607056, 0.7111849784851074, 0.7066434323787689, 0.7050520181655884, 0.7050461173057556, 0.6996228098869324, 0.6987706124782562, 0.6977443695068359, 0.6976523697376251]



Question: How do economic volatility in emerging markets, government policies like deferred loan payments, and temporary production halts during the pandemic collectively influence Honda's motorcycle and automotive business performance?



...


Question: How is the projected increase in leverage for companies involved in electric vehicle (EV) battery production expected to impact their Baa1 rating, and what risks do these companies face in the context of growing EV battery demand?



2024-10-16 12:15:29,141 - INFO - synthlume.pipeline.step.generate_question_with_enhanced_context - Most similar scores: [1.0, 0.8219613283872604, 0.8170081079006195, 0.8061583340167999, 0.8007654845714569, 0.7950980216264725, 0.7894247621297836, 0.7758824974298477, 0.7746129930019379, 0.7736906856298447, 0.7697441577911377, 0.7679910808801651, 0.761779859662056, 0.7612704634666443, 0.7554852366447449, 0.7538959383964539, 0.7526652216911316, 0.752517819404602, 0.7480019927024841, 0.7472227513790131]



Question: Considering the significant contribution of the EV battery business to the overall operating profit and the projected increase in leverage over the next one to two years, what are the key operational and investment risks associated with the growth in EV battery demand?


Question: How does the anticipated increase in leverage over the next one to two years and the global growth in EV battery demand influence the financial stability and risk management strategies of companies in the electric-vehicle battery industry?

Processing Chunk 5/5 using Regular Generation


2024-10-16 12:15:40,097 - INFO - synthlume.pipeline.step.generate_question_with_enhanced_context - Most similar scores: [0.9999700465668866, 0.8852085843682289, 0.8800028637051582, 0.8793438673019409, 0.876979410648346, 0.8756560608744621, 0.8745653629302979, 0.8741365373134613, 0.8713633716106415, 0.8710337728261948, 0.8676546812057495, 0.8641162216663361, 0.8473677337169647, 0.8463115841150284, 0.8461651802062988, 0.8426218926906586, 0.8419212996959686, 0.8416228294372559, 0.8407158553600311, 0.8301418870687485]
2024-10-16 12:15:45,217 - WARNING - synthlume.pipeline.step.json_step - Could not decode response as JSON, retrying (1/3)
2024-10-16 12:15:45,219 - WARNING - synthlume.pipeline.step.json_step - Error: Expecting value: line 1 column 1 (char 0)
2024-10-16 12:15:45,220 - WARNING - synthlume.pipeline.step.json_step - Response: ##RESPONSE##
Reasoning Steps:
1. Examine Context 1 for data on adjusted debt and EBITDA over the years, highlighting trends and anomalies.
2. Review Contex

Error processing chunk 5: 'question'


# Conclusion

### Comparative Analysis: Sampling Generation vs. Regular Generation

When comparing **sampling generation** and **regular generation** for question generation in LLMs, several key differences and insights can be drawn based on the depth, variety, and specificity of the generated questions. Below, we break down the comparative analysis using various criteria.

---

### 1. **Depth of the Questions**

**Sampling Generation:**
- **Examples:**  
   - *"How do economic recovery factors, including currency fluctuations, tourism dependency, and government policies, influence consumer demand for Honda's motorcycle and automotive products in emerging markets?"*
   - *"How does Tesla's competitive advantage in the electric vehicle market, particularly its operational efficiency and product lineup, mitigate the risks posed by increasing competition?"*

   **Analysis**: Sampling generation tends to produce more **multi-layered questions** that encompass multiple dimensions of the issue. For example, questions often include a combination of economic conditions, market dynamics, competitive positioning, and regulatory factors. These questions prompt deeper exploration and encourage a broader analysis of the relationships between various factors (e.g., "economic recovery," "consumer demand," and "government policies"). 

**Regular Generation:**
- **Examples:**  
   - *"How has Honda's motorcycle business been affected by both economic volatility in emerging markets and government policies during the pandemic?"*  
   - *"Considering the acceleration of electric vehicle initiatives by traditional automakers and Tesla's current market position, how might the competitive landscape impact Tesla's growth prospects?"*

   **Analysis**: Regular generation tends to produce more **focused questions** that address a specific aspect of the issue. The questions are often direct and clear, focusing on a single element like "motorcycle business" or "competitive landscape." While these questions are useful, they may lack the breadth seen in sampling-generated questions, making them slightly less exploratory.

**Conclusion**: Sampling generation produces more in-depth and multifaceted questions, providing greater insight into complex issues.

---

### 2. **Variety of Topics**

**Sampling Generation:**
- Covers a **wide range of topics** within the same domain, addressing various angles and interconnected factors. For instance, in the questions about Honda and Tesla, the approach touches on:
  - Economic conditions
  - Government policies
  - Market competition
  - Consumer behavior
  - Production strategies

   The variety also extends to different industries, such as US-China trade, automotive market dynamics, and electric vehicle battery demand. This breadth suggests that sampling generation better captures nuances in different contexts.

**Regular Generation:**
- Tends to stick to **key central themes** with less variation in the scope. For example, questions regarding Tesla focus mainly on competitive pressures and leverage but don’t explore as many factors like market shifts, regulatory environments, or specific financial outcomes.

**Conclusion**: Sampling generation provides greater variety, allowing for more comprehensive coverage of a topic from multiple angles.

---

### 3. **Specificity of Questions**

**Sampling Generation:**
- Often includes **more detailed and specific criteria** in the questions. For example:
   - *"How do Tesla's competitive advantage in battery electric vehicle production and regulatory support position the company to manage anticipated shifts in market dynamics due to increased vehicle offerings?"*
  
   These questions force the user to consider multiple interrelated factors (e.g., "competitive advantage," "battery production," and "regulatory support"), leading to a richer, more precise answer.

**Regular Generation:**
- Questions are more **general** and often focus on broader themes without delving into specifics. For example:
   - *"How has Honda's motorcycle business been affected by economic volatility in emerging markets?"*

   This structure allows for an answer that is straightforward but might lack the layers of insight required for a deep analysis.

**Conclusion**: Sampling generation offers greater specificity, prompting more detailed answers and fostering deeper insights.

---

### 4. **Complexity and Layering**

**Sampling Generation:**
- The questions often contain **multiple layers**, combining several dimensions like financial performance, market competition, government policies, and external risks. For instance, the question about Tesla's competitive positioning and operational efficiency asks for an analysis across several business areas.

**Regular Generation:**
- While still insightful, the questions generated without sampling tend to be **less complex**. They generally focus on one key issue, such as a company’s financial leverage or sales in specific regions, without necessarily requiring the same level of multi-dimensional analysis.

**Conclusion**: Sampling generation tends to generate more complex, layered questions that explore interconnected factors.

---

### 5. **Practicality and Usability**

**Sampling Generation:**
- These questions, while deeper and more insightful, may be **more challenging to answer** due to their complexity. In certain cases, they may require more resources (e.g., access to specific datasets or additional models) to answer comprehensively.

**Regular Generation:**
- Provides **more practical, easily answerable questions**, which are suitable for straightforward analyses. They are direct and can be addressed with less exploratory effort, making them more feasible in situations where time and resources are constrained.

**Conclusion**: Regular generation might be more practical for quick, direct analysis, while sampling generation is ideal for in-depth research requiring more exploration.

---

### 6. **Criteria for Choosing Between the Two**

- **Exploration and Insight Depth**: Choose **sampling generation** when the goal is to explore a topic deeply, uncover hidden patterns, or analyze complex relationships.
- **Efficiency and Simplicity**: Choose **regular generation** when you need straightforward, quick answers or are working within limited constraints.
- **Complex Analysis**: Sampling generation is better suited for research or business analysis where understanding the interplay of multiple factors is critical.
- **Breadth of Scope**: If you're looking for a broad range of angles on a single issue, sampling generation will likely provide a more varied set of questions.

---

### Final Verdict

**Sampling Generation** is superior in terms of depth, variety, and specificity. It produces questions that encourage complex and multi-dimensional analysis, which is crucial for in-depth explorations or when you're dealing with nuanced topics like market dynamics or financial health. 

**Regular Generation**, on the other hand, excels at producing clear, concise, and easy-to-answer questions, making it more practical for scenarios where simplicity and directness are needed.

